## ANSWERING BUSINESS QUESTIONS WITH SQL

The Chinook sample database is a good database for practicing with SQL, especially SQLite.

The Chinook data model represents a digital media store, including tables for artists, albums, media tracks, invoices, and customers.
Media-related data was created using real data from an Apple iTunes library.
Customer and employee information was created using fictitious names and addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.)
Sales information was auto generated using random data for a four year period.

In [ ]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## TOP 10 GENRES IN THE USA

In [79]:
%%sql 
WITH tracks_usa AS
                    (
                        SELECT 
                        g.name AS genre
                        FROM invoice_line il 
                        INNER JOIN invoice i ON il.invoice_id = i.invoice_id
                        INNER JOIN track t ON il.track_id = t.track_id
                        INNER JOIN genre g ON t.genre_id = g.genre_id
                        WHERE billing_country = 'USA'
                    ),
    total_tracks_usa AS
                    (
                        SELECT CAST(COUNT(*) AS float)
                        FROM tracks_usa
                    )

SELECT 
genre,
COUNT(*) AS number_of_tracks,
ROUND(100*COUNT(*)/(SELECT * FROM total_tracks_usa ), 1) AS percentages
FROM tracks_usa
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,number_of_tracks,percentages
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Pop,22,2.1
Latin,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


## ANALYZING EMPLOYEE SALES PERFORMANCE

In [88]:
%%sql
SELECT
e.first_name||' '||e.last_name AS employee,
e.hire_date,
ROUND(SUM(i.total), 2) AS total_sales
FROM invoice i
INNER JOIN customer c ON i.customer_id = c.customer_id
INNER JOIN employee e ON c.support_rep_id = e.employee_id
GROUP BY c.support_rep_id


 * sqlite:///chinook.db
Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


## ANALYZING SALES BY COUNTRY

In [165]:
%%sql
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT 
    country,
    customers,
    total_sales,
    avg_order_value,
    avg_sales_value_per_customer
FROM 
    (
      SELECT
          country,
          COUNT(DISTINCT(customer_id)) AS customers,
          SUM(unit_price) AS total_sales,
          SUM(unit_price) / COUNT(DISTINCT(customer_id)) AS avg_sales_value_per_customer,
          SUM(unit_price) / COUNT(DISTINCT(invoice_id))AS avg_order_value,
      CASE
            WHEN country = 'Other' THEN 1
            ELSE 0
      END AS sort
      FROM country_or_other
      GROUP BY country
      ORDER BY sort ASC, total_sales DESC
    );


 * sqlite:///chinook.db
Done.


country,customers,total_sales,avg_order_value,avg_sales_value_per_customer
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


## ALBUMS vs INDIVIDUAL TRACKS

In [219]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;



 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674
